In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random

from tensorflow.keras.layers import Dense, InputLayer,Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
import pickle
import keras


In [2]:
img_size = 100

In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
for layer in base_model.layers:
    layer.trainable = False

In [4]:
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', ])

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Traning data
train_data_dir = 'chest_xray/train'
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_size, img_size), batch_size=4, class_mode='binary', shuffle=True)    

Found 5216 images belonging to 2 classes.


In [7]:
# Validation data
val_data_dir = 'chest_xray/val' 
val_generator = val_datagen.flow_from_directory(val_data_dir, target_size=(img_size, img_size), batch_size=4, class_mode='binary', shuffle=True)


Found 16 images belonging to 2 classes.


In [8]:
# Testing data
tets_data_dir = 'chest_xray/test' 
test_generator = test_datagen.flow_from_directory(tets_data_dir, target_size=(img_size, img_size), batch_size=4, class_mode='binary', shuffle=False)

Found 624 images belonging to 2 classes.


In [9]:
# Training the model
history = model.fit(train_generator,epochs=10, validation_data=val_generator)

Epoch 1/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 105s 80ms/step - accuracy: 0.9103 - loss: 0.2192 - val_accuracy: 0.7500 - val_loss: 0.4164
Epoch 2/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 104s 80ms/step - accuracy: 0.9390 - loss: 0.1598 - val_accuracy: 0.8125 - val_loss: 0.3649
Epoch 3/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 107s 82ms/step - accuracy: 0.9456 - loss: 0.1468 - val_accuracy: 0.8750 - val_loss: 0.2258
Epoch 4/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 106s 81ms/step - accuracy: 0.9505 - loss: 0.1324 - val_accuracy: 0.7500 - val_loss: 0.3449
Epoch 5/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 105s 80ms/step - accuracy: 0.9519 - loss: 0.1311 - val_accuracy: 0.9375 - val_loss: 0.2756
Epoch 6/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 111s 85ms/step - accuracy: 0.9521 - loss: 0.1312 - val_accuracy: 0.8750 - val_loss: 0.1940
Epoch 7/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 106s 81ms/step - accuracy: 0.9530 - loss: 0.1235 - val_accuracy: 0.8125 - val_loss: 0.2856
Epoch 8/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 104s 80ms/step - accuracy: 

In [13]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

156/156 ━━━━━━━━━━━━━━━━━━━━ 11s 69ms/step - accuracy: 0.9231 - loss: 0.2186
Test accuracy: 0.9230769276618958


In [12]:
model_name = f'Model/Pneumonia_Detection_VGG_{round(test_acc * 100, 0)}.h5'
tf.keras.models.save_model(model, model_name)